This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Naive Interpolation of Euler Angles

This method for interpolating 3D rotations is very much not recommended!

Since 3D rotations can be represented by a list of three angles,
it might be tempting to simply interpolate those angles independently.

Let's try it and see what happens, shall we?

In [ ]:
import numpy as np

In [ ]:
import splines

As always, we use a few helper functions from [helper.py](helper.py):

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

We are using
[splines.CatmullRom](../python-module/splines.rst#splines.CatmullRom)
to interpolate the Euler angles independently and
[splines.quaternion.CatmullRom](../python-module/splines.quaternion.rst#splines.quaternion.CatmullRom)
to interpolate the associated quaternions for comparison:

In [ ]:
def plot_interpolated_angles(angles):
    s1 = splines.CatmullRom(angles, endconditions='closed')
    times = np.linspace(s1.grid[0], s1.grid[-1], 100)
    s2 = splines.quaternion.CatmullRom(
        [angles2quat(azi, ele, roll) for azi, ele, roll in angles],
        endconditions='closed')
    ani = animate_rotations({
        'Euler angles': [angles2quat(*abc) for abc in s1.evaluate(times)],
        'quaternions': s2.evaluate(times),
    }, figsize=(6, 3))
    display_animation(ani, default_mode='loop')

In [ ]:
plot_interpolated_angles([
    (0, 0, 0),
    (45, 0, 0),
    (90, 45, 0),
    (90, 90, 0),
    (180, 0, 90),
])

There is clearly a difference between the two,
but the Euler angles don't look that bad.

Let's try another example:

In [ ]:
plot_interpolated_angles([
    (-175, 0, 0),
    (175, 0, 0),
])

Here we see that the naive interpolation isn't aware
that the azimuth angle is supposed to wrap around at 180 degrees.

This could be fixed with a less naive implementation,
but there are also unfixable problems,
as this example shows:

In [ ]:
plot_interpolated_angles([
    (45, 45, 0),
    (45, 90, 0),
    (-135, 45, 180),
])

Even though all involved rotations are supposed to happen
around a single rotation axis,
The Euler angles interpolation is all over the place.